In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%pip install sympy 
from sympy import *

Note: you may need to restart the kernel to use updated packages.


In [ ]:
df_b = pd.read_csv('data/BBRs_5538500_5538999.csv')


In [ ]:
df_p = pd.read_csv('data/PPDs_5538500_5540000.csv')
df_p

In [ ]:
df_p.drop_duplicates(subset=['slot'], keep='first', inplace=True)

In [ ]:
df_p.sort_values(by=['slot'], ascending=True, inplace=True)
df_p.rename(columns={'buidler_pubkey': 'builder_pubkey'}, inplace=True)


In [ ]:
df_p = df_p[['slot', 'block_hash', 'block_reward', 'gas_used', 'validator_pubkey', 'builder_pubkey', 'builder_name', 'relay_name']]


In [ ]:
slot = np.arange(5538500,5540000)
df = pd.DataFrame(slot, columns = ['slot'])

In [ ]:
df = df.merge(df_p, how='left', on='slot')
df

In [ ]:
df['count'] = 1
df['position'] = 0
count = 0
for i in range(0, len(df)):
    #check = df['builder_pubkey'].iloc[i] == df['builder_pubkey'].iloc[i-1] 
    check = (df['builder_name'].iloc[i] == df['builder_name'].iloc[i-1]) & (df['builder_name'].iloc[i] != 'Unknown')
    
    if check == True:
        if count == 0:
            count = 2
        else:
            count += 1

    elif (check == False) & (count == 0):
            continue

    elif (check == False) & (count != 0):
        for j in range(1, count+1):
            df['count'].iloc[i-j] = count
            df['position'].iloc[i-j] = count+1-j

        count = 0
        
        

In [ ]:
df['block_reward'].fillna((df[df['count'] == 1]['block_reward'].mean()), inplace=True)

In [ ]:
data_1 = list(df[df['count']==1]['block_reward'])
data_2 = list(df[df['count']==2]['block_reward'])
data_3 = list(df[df['count']==3]['block_reward'])
data_4 = list(df[df['count']==4]['block_reward'])
data_5 = list(df[df['count']==4]['block_reward'])
data = [data_1, data_2, data_3, data_4, data_5]
 
fig, ax = plt.subplots(figsize =(8, 8))

ax.boxplot(data, showfliers=False)

ax.set_ylabel('Mean builder payment')
ax.set_xlabel('Consecutive blocks')

vals = ax.get_yticks()
ax.set_yticklabels(['{:,.2f}'.format(x)+' ETH' for x in vals])

# show plot
plt.show()


In [ ]:
stat = df.groupby(df['count']).size().values
stat[1] = stat[1]/2
stat[2] = stat[2]/3
stat[3] = stat[3]/4
stat[4] = stat[4]/5

print('In a dataset of', len(df), 'blocks (5h), we find:')
print(stat[0], 'single blocks')
print(stat[1], 'instances of 2 consecutive blocks')
print(stat[2], 'instances of 3 consecutive blocks')
print(stat[3], 'instances of 4 consecutive blocks')
print(stat[4], 'instances of 5 consecutive blocks')
print('proposed by the same builder')

In [ ]:
df[df['count'] >3].head(50)

In [ ]:
fig, ax = plt.subplots(figsize =(8, 8))


ax.plot(df['position'].unique(), df['block_reward'].groupby(df['position']).mean())

ax.set_ylabel('Mean builder payment')
ax.set_xlabel('n position in consecutive blocks')

vals = ax.get_yticks()
ax.set_yticklabels(['{:,.2f}'.format(x)+' ETH' for x in vals])

# show plot
plt.show()


array([0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 ])

In [79]:
def prob(r, p):
    # r = consecutive blocks
    # n = slots
    # p = share MEV boost
    
    x = symbols('x', real=True)

    r = r; n = 32; p = p

    xx = min(i for i in solve(1-x+(1-p)*p**r*x**(r+1)) if i > 1)
    qn = ((1 - p*xx) / ((r + 1 - r*xx)*(1-p))) * (1/(xx**(n+1)))
    
    return qn*100, (1-qn)*100

    

In [78]:
cons_blocks = np.arange(2,10+1)
p = np.arange(.5,.9+.05,.05)

In [80]:
lst = []
for x in p:
    lst_temp = []
    lst_temp.append(x)
    for y in cons_blocks:
        no_success, success = prob(y, x)
        lst_temp.append(success)
    
    for z in range(0, len(lst_temp)):
        try: 
            lst_temp[z] = np.round(float(lst_temp[z]), 2)
        except TypeError:
            lst_temp[z] = np.round(float(100), 2)
        
    lst.append(lst_temp)
        

In [81]:
df = pd.DataFrame(lst, columns =['probability', 'k=2', 'k=3', 'k=4', 'k=5', 'k=6', 'k=7', 'k=8', 'k=9', 'k=10'])
df = df.set_index('probability')
df

,k=2,k=3,k=4,k=5,k=6,k=7,k=8,k=9,k=10
probability,,,,,,,,,
0.50,99.87,92.21,66.47,38.96,20.50,10.26,5.02,2.43,1.17
0.55,99.97,96.50,78.66,52.94,31.63,17.75,9.66,5.17,2.75
0.60,99.99,98.68,88.07,67.06,45.21,28.35,17.09,10.07,5.87
0.65,100.00,99.60,94.33,79.64,60.09,41.94,27.91,18.06,11.49
0.70,100.00,99.91,97.81,89.31,74.45,57.52,42.16,29.87,20.71
0.75,100.00,100.00,99.36,95.50,86.33,73.15,58.82,45.54,34.35
0.80,100.00,100.00,100.00,98.62,94.35,86.32,75.53,63.68,52.18
0.85,100.00,100.00,100.00,100.00,98.44,95.04,89.13,81.14,71.94
0.90,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,89.01


In [82]:
print('If the market share of MEV boost is p=0.5, then we derive', (38.96/100)*225, 'k=5 consecutive blocks a day.')
print('If the market share of MEV boost is p=0.5, then we derive', (38.96/100)*6759, 'k=5 consecutive blocks a month.')


If the market share of MEV boost is p=0.5, then we derive 87.66 k=5 consecutive blocks a day.
If the market share of MEV boost is p=0.5, then we derive 2633.3064 k=5 consecutive blocks a month.
